# Exploratory Analysis of NUTS 2 GDP Values

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

In [4]:
data = pd.read_csv("/Users/maxbehrens/Documents/Msc/Thesis/Data/gdp_data/nuts_gdp_cleaned.csv")
print(data.shape)
print(data.head(10))

(4179, 5)
  region  year    value code  country
0     BE  2008  33100.0   BE  Belgium
1    BE1  2008  61100.0   BE  Belgium
2   BE10  2008  61100.0   BE  Belgium
3    BE2  2008  33100.0   BE  Belgium
4   BE21  2008  39200.0   BE  Belgium
5   BE22  2008  27500.0   BE  Belgium
6   BE23  2008  29400.0   BE  Belgium
7   BE24  2008  34500.0   BE  Belgium
8   BE25  2008  31100.0   BE  Belgium
9    BE3  2008  24400.0   BE  Belgium


In [44]:
# Import GDP PPP data
ppp_data = pd.read_excel("/Users/maxbehrens/Documents/Msc/Thesis/Data/gdp_data/gdp_ppp.xls",na_values=':')
print(ppp_data.head())

geo   2006   2007   2008   2009   2010   2011   2012   2013   2014   2015  \
0  BE10  228.0  219.0  213.0  219.0  220.0  217.0  216.0  212.0  205.0  204.0   
1  BE21  140.0  138.0  137.0  137.0  141.0  140.0  142.0  141.0  141.0  140.0   
2  BE22   97.0   96.0   96.0   94.0   97.0   98.0   99.0   98.0   99.0   97.0   
3  BE23  105.0  104.0  102.0  107.0  108.0  108.0  109.0  109.0  109.0  109.0   
4  BE24  124.0  122.0  120.0  126.0  127.0  125.0  130.0  130.0  128.0  127.0   

    2016   2017  
0  201.0  196.0  
1  140.0  140.0  
2   96.0   96.0  
3  109.0  108.0  
4  128.0  125.0  


In [45]:
# Reshape to long format
ppp_long = pd.melt(ppp_data, id_vars=['geo'],value_vars=['2008','2009','2010','2011','2012','2013','2014','2015','2016','2017'])
ppp_long = ppp_long.dropna()
ppp_long.head()

,geo,variable,value
0,BE10,2008,213.0
1,BE21,2008,137.0
2,BE22,2008,96.0
3,BE23,2008,102.0
4,BE24,2008,120.0


In [16]:
# Average regions per country
print("Average regions per country: ",data.groupby(['country','year']).region.count().reset_index().region.mean())
print("Min regions per country: ",data.groupby(['country','year']).region.count().reset_index().region.min())
print("Max regions per country: ",data.groupby(['country','year']).region.count().reset_index().region.max())

Average regions per country:  13.39423076923077
Min regions per country:  3
Max regions per country:  55


## Benchmark Prediction for GDP Values

In [17]:
country_values = data[data['region'].str.len() == 2]
country_values.head()

,region,year,value,code,country
0,BE,2008,33100.0,BE,Belgium
15,BG,2008,4900.0,BG,Bulgaria
24,CZ,2008,15500.0,CZ,Czechia
34,DK,2008,44000.0,DK,Denmark
41,DE,2008,31700.0,DE,Germany


In [21]:
# Open list of NUTS 2 regions
with open("/Users/maxbehrens/Documents/Msc/Thesis/Data/geo_data_2016/NUTS_RG_01M_2016_4326_LEVL_2.geojson") as f:
    nuts2_2016 = json.load(f)

# Open list of NUTS 2 regions
with open("/Users/maxbehrens/Documents/Msc/Thesis/Data/geo_data_2013/NUTS_RG_01M_2013_4326_LEVL_2.geojson") as f:
    nuts2_2013 = json.load(f)

# Open list of NUTS 2 regions
with open("/Users/maxbehrens/Documents/Msc/Thesis/Data/geo_data_2010/NUTS_RG_01M_2010_4326_LEVL_2.geojson") as f:
    nuts2_2010 = json.load(f)

# Open list of NUTS 2 regions
with open("/Users/maxbehrens/Documents/Msc/Thesis/Data/geo_data_2006/NUTS_RG_01M_2006_4326_LEVL_2.geojson") as f:
    nuts2_2006 = json.load(f)

In [23]:
# Make list of all downloaded regions, which will be predicted
nuts2_regions = []
for i in range(len(nuts2_2016['features'])): 
    nuts2_regions.append(nuts2_2016['features'][i]['properties']['NUTS_ID'])
for i in range(len(nuts2_2013['features'])): 
    nuts2_regions.append(nuts2_2013['features'][i]['properties']['NUTS_ID'])
for i in range(len(nuts2_2010['features'])): 
    nuts2_regions.append(nuts2_2010['features'][i]['properties']['NUTS_ID'])
for i in range(len(nuts2_2006['features'])): 
    nuts2_regions.append(nuts2_2006['features'][i]['properties']['NUTS_ID'])

# Check
print(len(nuts2_regions))

1286


In [24]:
# Filter for predicted regions
nuts2_predictions = data[data['region'].isin(nuts2_regions)]
print(nuts2_predictions.shape)
print(nuts2_predictions.head())

(2827, 5)
  region  year    value code  country
2   BE10  2008  61100.0   BE  Belgium
4   BE21  2008  39200.0   BE  Belgium
5   BE22  2008  27500.0   BE  Belgium
6   BE23  2008  29400.0   BE  Belgium
7   BE24  2008  34500.0   BE  Belgium


## Predict values with total value of country and year

In [25]:
# Merge with total values per country
country_predictions = nuts2_predictions.merge(country_values.loc[:,['region','year','value']], left_on=['code','year'], right_on=['region','year']).drop(columns = ['region_y'])
country_predictions.columns = ['nuts2','year','nuts_value','code','country','country_value']
country_predictions.head()

,nuts2,year,nuts_value,code,country,country_value
0,BE10,2008,61100.0,BE,Belgium,33100.0
1,BE21,2008,39200.0,BE,Belgium,33100.0
2,BE22,2008,27500.0,BE,Belgium,33100.0
3,BE23,2008,29400.0,BE,Belgium,33100.0
4,BE24,2008,34500.0,BE,Belgium,33100.0


In [26]:
# Compute MAE and MSE for predictions
mae_country_value = np.mean(abs(country_predictions['country_value']-country_predictions['nuts_value']))
mse_country_value = np.mean((country_predictions['country_value']-country_predictions['nuts_value'])**2)
print('MAE: ', mae_country_value)
print('MSE: ', mse_country_value)

MAE:  5889.246551114255
MSE:  137490948.00141492


## Predict values with total mean

In [27]:
# Compute MAE and MSE for predictions
mae_total_value = np.mean(abs(np.mean(country_predictions['nuts_value'])-country_predictions['nuts_value']))
mse_total_value = np.mean((np.mean(country_predictions['nuts_value'])-country_predictions['nuts_value'])**2)
print('MAE: ', mae_total_value)
print('MSE: ', mse_total_value)

MAE:  11795.816854729313
MSE:  292515102.45398825


## Predict values by the mean of regions per country and year

In [28]:
mean_predictions = country_predictions.groupby(['code','year'])['nuts_value'].mean().reset_index()
mean_predictions.head()

,code,year,nuts_value
0,AL,2008,2933.333333
1,AL,2009,2933.333333
2,AL,2010,3066.666667
3,AL,2011,3166.666667
4,AL,2012,3233.333333


In [29]:
mean_predictions = country_predictions.merge(mean_predictions, left_on=['code','year'], right_on=['code','year'])
mean_predictions.columns = ['nuts2','year','nuts_value','code','country','country_value','mean_value']
mean_predictions.head()

,nuts2,year,nuts_value,code,country,country_value,mean_value
0,BE10,2008,61100.0,BE,Belgium,33100.0,31881.818182
1,BE21,2008,39200.0,BE,Belgium,33100.0,31881.818182
2,BE22,2008,27500.0,BE,Belgium,33100.0,31881.818182
3,BE23,2008,29400.0,BE,Belgium,33100.0,31881.818182
4,BE24,2008,34500.0,BE,Belgium,33100.0,31881.818182


In [30]:
# Compute MAE and MSE for predictions
mae_mean_value = np.mean(abs(mean_predictions['nuts_value']-mean_predictions['mean_value']))
mse_mean_value = np.mean((mean_predictions['nuts_value']-mean_predictions['mean_value'])**2)
print('MAE: ', mae_mean_value)
print('MSE: ', mse_mean_value)

MAE:  5280.572333692404
MSE:  127639832.55490972


## Add another column with ppp value

In [32]:
print(ppp_long.shape)
print(mean_predictions.shape)

(2827, 3)
(2827, 7)


In [47]:
ppp_long['variable'] = pd.to_numeric(ppp_long['variable'])

In [48]:
enhanced_gdp = mean_predictions.merge(ppp_long, how='left', left_on=['nuts2','year'], right_on=['geo','variable'])
print(enhanced_gdp.head)
print(enhanced_gdp.shape)

<bound method NDFrame.head of      nuts2  year  nuts_value code  country  country_value    mean_value   geo  \
0     BE10  2008     61100.0   BE  Belgium        33100.0  31881.818182  BE10   
1     BE21  2008     39200.0   BE  Belgium        33100.0  31881.818182  BE21   
2     BE22  2008     27500.0   BE  Belgium        33100.0  31881.818182  BE22   
3     BE23  2008     29400.0   BE  Belgium        33100.0  31881.818182  BE23   
4     BE24  2008     34500.0   BE  Belgium        33100.0  31881.818182  BE24   
...    ...   ...         ...  ...      ...            ...           ...   ...   
2822  TRB1  2017      5600.0   TR   Turkey         9400.0   7665.384615  TRB1   
2823  TRB2  2017      3700.0   TR   Turkey         9400.0   7665.384615  TRB2   
2824  TRC1  2017      6200.0   TR   Turkey         9400.0   7665.384615  TRC1   
2825  TRC2  2017      3900.0   TR   Turkey         9400.0   7665.384615  TRC2   
2826  TRC3  2017      4500.0   TR   Turkey         9400.0   7665.384615  TRC3  

In [50]:
enhanced_gdp = enhanced_gdp.drop(columns = ['geo','variable'])
enhanced_gdp.columns = ['nuts2', 'year', 'nuts_value', 'code', 'country', 'country_value',
       'mean_value','ppp']
print(enhanced_gdp.head())

nuts2  year  nuts_value code  country  country_value    mean_value    ppp
0  BE10  2008     61100.0   BE  Belgium        33100.0  31881.818182  213.0
1  BE21  2008     39200.0   BE  Belgium        33100.0  31881.818182  137.0
2  BE22  2008     27500.0   BE  Belgium        33100.0  31881.818182   96.0
3  BE23  2008     29400.0   BE  Belgium        33100.0  31881.818182  102.0
4  BE24  2008     34500.0   BE  Belgium        33100.0  31881.818182  120.0


In [52]:
enhanced_gdp.to_csv("/Users/maxbehrens/Documents/Msc/Thesis/Data/gdp_data/enhanced_gdp_data.csv", index=False)